# ViT vs CNN

In [8]:
import json
import requests
import io
import glob

import BiT
from ViT import modeling as ViT

import PIL
from PIL import Image
from PIL.Image import Image as Img
import matplotlib.pyplot as plt
import numpy as np

from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision as tv
import torchvision.transforms as T

In [9]:
dataset_path = 'images'

In [10]:
def get_weights(path):
  return np.load(path)

## Prepare dataset and model

In [11]:
IMG_SIZE = (384, 384)
NORMALIZE_MEAN = (0.5, 0.5, 0.5)
NORMALIZE_STD = (0.5, 0.5, 0.5)
transforms = [
              T.Resize(IMG_SIZE),
              T.ToTensor(),
              T.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
              ]

transforms = T.Compose(transforms)

In [30]:
def load_images(dataset_path: str) -> List[Img]:
    images = []
    for filename in glob.glob(dataset_path + '/*'):
        im=Image.open(filename).convert('RGB')
        images.append(im)
    return images


def load_vit(model_name='ViT-H_14') -> nn.Module:
    config = ViT.CONFIGS[model_name]
    model = ViT.VisionTransformer(config, img_size=384, vis=True)
    model.load_from(get_weights('imagenet21k-' + model_name + '.npz'))
    return model
    
def load_bit(model_name='BiT-M-R152x4') -> nn.Module:
    model = BiT.KNOWN_MODELS[model_name]()
    model.load_from(get_weights(model_name + '.npz'))
    return model

In [33]:
images = load_images(dataset_path)
vit = load_vit()
bit = load_bit()

img = images[0]
img_tensor = transforms(img).unsqueeze(0)
out_vit = vit(img_tensor)
out_bit = bit(img_tensor)

load_pretrained: grid-size from 16 to 27


## Compare predictions

In [37]:
assert np.allclose(out_vit[0].detach().numpy(), out_bit.detach().numpy()), '\033[93m' 'Failed test!'

print('\033[92m' 'Passed test!')

AssertionError: [93mFailed test!

## Compare activation maps

In [ ]:
# TODO

## Compare embeddings clusters

In [ ]:
# TODO